In [9]:
import numpy as np  
from asteroestimate import detections
from asteroestimate.detections import probability as prob                                               
from scipy.stats import norm, multivariate_normal 
class NuPrior(object):
    '''                                                                                              
    Provide guesses for numax using three different methods and also optionally numax prior distributions.
    1) specnmx()
     Uses spectroscopic log g + spectroscopic temperature.
    2) gaiascalnmx()
     Uses Gaia parallax + apparent magnitude + bolometric correction + photometric temperature + optional extinction.
    3) gaiamlnmx():
     Uses a data-driven approach to map Gaia luminosity to numax.
    '''
    
    def __init__(plx=None, plx_err=None, logg_spec=None, logg_spec_err=None, teff_spec=None, teff_spec_err=None,
                 jmag=None, jmag_err=None, hmag=None, hmag_err=None, kmag=None, kmag_err=None):
        ''' 
        INPUTS:                                                                                              
        [ plx, plx_err : float, float ]
         Parallax and uncertainty. Default None.
        [ logg_spec, logg_spec_err : float, float ]
         Spectroscopic log g and uncertainty. Default None.
        [ teff_spec, teff_spec_err : float, float ]
         Spectroscopic temperature and uncertainty. Default None.  
        [ jmag, jmag_err : float, float ]
         J-band magnitude and uncertainty. Default None.
        [ hmag, hmag_err : float, float ]
         H-band magnitude and uncertainty. Default None. 
        [ kmag, kmag_err : float, float ]
         K-band magnitude and uncertainty. Default None.
        HISTORY:                                                                                            
        Created 8 sep 20
        Joel Zinn (j.zinn@unsw.edu.au)
        '''
    
    def numax_JHK(mass=1., AK=None, N_samples=1000):                                     
        """                                                                                                 
        Evaluate a prior on numax based on 2MASS magnitudes and Gaia parallax                               
        INPUTS:                                                                                              
        [ plx, plx_err, jmag, jmag_err, hmag, hmag_err, kmag, kmag_err ] : [ float, float, float, float, float, float, float, float ]
         These need to be defined in __init__().
        [ mass : float ]
         Optional mass prior option (not yet implemented!!!). Default 1.               
        [ AK : float ]
         Optional K band extinction. Default None.                                                               
        [ N_samples : int ]
         Number of samples from the prior to take and then return. Default 1000.        
        OUTPUTS:                                                                                             
        out - summary statistics assuming samples are Gaussian distributed (numax_mean, numax_sigma)    
        samples - samples from the prior                                                                
        HISTORY:                                                                                            
        Written - Mackereth - 08/09/2020 (UoB @ online.tess.science)
        Modified JCZ 8 sep 20
        """                                                                                                 
        means = np.array([self.jmag, self.hmag, self.kmag, self.plx])                                                   
        cov = np.zeros((4,4))                                                                               
        cov[0,0] = self.jmag_err**2                                                                                  
        cov[1,1] = self.hmag_err**2                                                                                  
        cov[2,2] = self.kmag_err**2                                                                                  
        cov[3,3] = self.plx_err**2                                                                           
        multi_norm = multivariate_normal(means, cov)                                                        
        samples = multi_norm.rvs(size=N_samples)                                                            
        Jsamp, Hsamp, Ksamp, parallaxsamp = samples[:,0], samples[:,1], samples[:,2], samples[:,3]          
        numaxsamp = prob.numax_from_JHK(Jsamp, Hsamp, Ksamp, parallaxsamp, mass=mass, AK=AK)                
        numax_mean = np.mean(numaxsamp)                                                                     
        numax_sigma = np.std(numaxsamp)                                                                     
        return (numax_mean, numax_sigma), numaxsamp                                                         

def get_gaiascalnmx():
    nup = NuPrior(plx=1.0, plx_err=0.01, jmag=10., jmag_err=0.01, hmag=12., hmag_err=0.01, kmag=11., kmag_err=0.01)
    print('(numax_mean, numax_sigma, numax_amp) from gaiascalnmx:')
    print(nup.numax_JHK())
get_gaiascalnmx()  

    

ImportError: cannot import name 'detections'